# REST API Examples

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `API_URL` to the location of the REST API.

In [1]:
import pandas as pd
import requests
API_URL = "https://mastapp.site/json"

Below we print the location of the endpoint we will query in this notebook:

In [2]:
print(f"REST API Endpoint: {API_URL}")

REST API Endpoint: https://mastapp.site/json


## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [3]:
response = requests.get(f'{API_URL}/shots')
result = response.json()
print(f"Query returned status code: {response.status_code}")

Query returned status code: 200


The shots endpoint returns a JSON payload with a list of shots. Let's look at the first element from the payload:

In [4]:
result['items'][0]

{'title': 'Shot Dataset',
 'shot_id': 11695,
 'uuid': '219e5d59-b9dc-584f-b83b-018c47e8739d',
 'url': 's3://mast/level1/shots/11695.zarr',
 'endpoint_url': 'https://s3.echo.stfc.ac.uk',
 'timestamp': '2004-12-13T11:54:00',
 'preshot_description': '\n0.1T TF SHOT\n',
 'postshot_description': '\nOK\n',
 'campaign': 'M5',
 'reference_shot': None,
 'scenario': None,
 'heating': None,
 'pellets': False,
 'rmp_coil': None,
 'mcs_gdc_pre_shot': None,
 'mcs_select_gas_group_pv1': None,
 'mcs_select_gas_group_pv2': None,
 'mcs_select_gas_group_pv3': None,
 'mcs_select_gas_group_pv4': None,
 'mcs_select_gas_group_pv5': None,
 'mcs_select_gas_group_pv6': None,
 'mcs_select_gas_group_pv7': None,
 'mcs_select_gas_group_pv8': None,
 'mcs_select_gas_group_pv9': None,
 'mcs_select_lvps_tf_power_supply': None,
 'mcs_select_cp3s_start_bank': None,
 'mcs_select_fa1': None,
 'mcs_select_fa2': None,
 'mcs_select_mfps': None,
 'mcs_select_efps': None,
 'mcs_select_sfps': None,
 'mcs_select_scs4': None,
 'mc

Each item in the list is a json object. This contains the meta-data items that corresponded to our query. In this case, each item contains information about a different MAST shot. Each item has lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://mast-app.site/redoc


Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe directly from the endpoint data

In [5]:
df = pd.read_json(f'{API_URL}/shots')['items']
df = pd.DataFrame(df.to_list())
df.head()

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.

### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned. For example, we can return all shots for the `M9` campaign by using the approrpiate query string.

For example, we can query for everything from the `M9` campaign by adding `?campaign=M9` to our query string.

In [6]:
df = pd.read_json(f"{API_URL}/shots?campaign=M9")['items']
df = pd.DataFrame(df.to_list())
df.head()

,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,15085,0bb8ea1b-954b-5cce-8b2d-df40a0703b2f,s3://mast/level1/shots/15085.zarr,2006-04-25T13:48:00,\nReplace ccbv connector. repeat.\n,\nRadial control unchanged. Zip/FA jump less ...,M6,15084.0,None,None,...,None,None,None,None,None,None,None,None,None,None
1,15086,2ac3c2b7-0444-5f3b-86a9-38ef6bb3c160,s3://mast/level1/shots/15086.zarr,2006-04-25T14:15:00,\nAdd another 5cm to Drref during the ramp. Re...,"\nEarly termination, at 180ms.\n",M6,15085.0,None,None,...,None,None,None,None,None,None,None,None,None,None
2,15087,9867930b-af84-5866-a62c-9fe77d9a3def,s3://mast/level1/shots/15087.zarr,2006-04-25T14:34:00,"\nRepeat, with small change to IDIV.\n",\nEarly termination at 200ms.\n,M6,15086.0,None,None,...,None,None,None,None,None,None,None,None,None,None
3,15088,f3f7ffe4-bfd1-5b8d-8856-9d7e4377724b,s3://mast/level1/shots/15088.zarr,2006-04-25T14:52:00,\nRestore shot 15085.\n,\nOK.\n,M6,15085.0,None,None,...,None,None,None,None,None,None,None,None,None,None
4,15089,1adae615-46c4-5262-a49d-b25f707b0857,s3://mast/level1/shots/15089.zarr,2006-04-25T15:17:00,"\nRepeat, with IDIV, BV tweaked to match last ...",\nTerminated early.\n,M6,15088.0,None,None,...,None,None,None,None,None,None,None,None,None,None


### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response and corresponds to [RFC 8288](https://datatracker.ietf.org/doc/html/rfc8288). The response contains cursors at the bottom of the response:

 - `current_page` - this is the cursor relating to the current page of results.
 - `next_page` - this is the cursor for the next page of results.
 - `previous_page` - this is the cursor for the previous page of results, will result in `null` if on the first page.

You can control the pagination using the following options as query arguments:
 - `cursor=xx` to set the cursor for the displayed set of results
 - `size=xx` to set the number of results returned by each page.

In [7]:
response = requests.get(f'{API_URL}/shots?size=2&cursor=Pmk6MzAxMTE%3D')
result = response.json()
headers = response.headers

print("Current page cursor", result['current_page'])
print("Next page cursor", result['next_page'])
print("Previous page cursor", result['previous_page'])

df = pd.DataFrame(result['items'])
df

Current page cursor Pmk6MzAxMTE%3D
Next page cursor Pmk6MzAxMTM%3D
Previous page cursor PGk6MzAxMTI%3D


,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,30112,eefe2467-69b0-5fe7-9863-e21289daa135,s3://mast/level1/shots/30112.zarr,2013-09-09T11:42:00,\nReload 30110 - reduce IP flat top to 750 KA ...,\nPIC happy with data.\n,M9,30110,4,SS Beam,...,7.845118,8.841636,0.0,65898.305250,82056.664,0.0,None,None,None,0.047497
1,30113,742c78fa-3811-5404-93b3-5ab419d330bd,s3://mast/level1/shots/30113.zarr,2013-09-09T12:00:00,\nRestore 30111\n,\nShot OK for programme. Two good beams. Big l...,M9,30111,3,"2 Beams,SS Beam,SW Beam",...,8.391672,8.426641,0.0,43127.703046,60260.926,0.0,None,None,None,0.042220
